In [1]:
from selenium import webdriver

driver = webdriver.Chrome('/Users/lana/Desktop/projects/chromedriver')
driver.get('https://www.aladin.co.kr/home/welcome.aspx')
driver.find_element_by_xpath('')

In [5]:
import requests
from bs4 import BeautifulSoup

In [104]:
book_name = '알고리즘 기초 도경구'
req = requests.get('https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=All&SearchWord='+ book_name +'&x=0&y=0')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
book_link = [link.get('href') for link in soup.select('table > tbody > tr > td > div > ul > li > a')]

#Search3_Result > div > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > ul > li:nth-child(2) > a
#Search3_Result > div > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(1) > a

In [105]:
book_link

[]

### 크롤링 순서
1. ISBN 넘버가 첫 도서 검색시에 나오지 않는다. 도서 상세 페이지로 들어가야 나온다.
2. 도서 상세 페이지 링크를 우선 수집한다.
3. 상세 링크 리스트를 만든다.
4. 리스트 for문을 돌면서 다시 ISBN만 수집하는 request함수를 짠다. 

In [52]:
import pandas as pd

검색할 도서 리스트 불러오기

In [53]:
booklist = pd.read_csv('./booklist.csv')

In [55]:
booklist.head()

,ISBN,분류,도서명,저자,출판사명,도서가,수량
0,NaN,IT,Mano의 컴퓨터시스템구조,김종상,프로텍미디어,"33,000",2
1,NaN,IT,데이터베이스 시스템,이석호,정익사,"36,000",2
2,NaN,IT,C로 쓴 자료구조,이석호 역,교보문고,"28,000",2
3,NaN,IT,C언어로 쉽게 풀어쓴 자료구조,"천인국, 공용해, 하상호",생능출판사,"29,000",2
4,NaN,IT,데이터통신과 네트워킹,Forouzan Behrouz A.,McGraw-Hill,"39,000",2


도서명만 검색하면 안나오는 경우가 있기 때문에 저자를 함께 검색합니다. 

In [59]:
booklist['key_value'] = booklist['도서명'] + ' ' + booklist['저자']

In [67]:
booklist

,ISBN,분류,도서명,저자,출판사명,도서가,수량,key_value
0,NaN,IT,Mano의 컴퓨터시스템구조,김종상,프로텍미디어,"33,000",2,Mano의 컴퓨터시스템구조 김종상
1,NaN,IT,데이터베이스 시스템,이석호,정익사,"36,000",2,데이터베이스 시스템 이석호
2,NaN,IT,C로 쓴 자료구조,이석호 역,교보문고,"28,000",2,C로 쓴 자료구조 이석호 역
3,NaN,IT,C언어로 쉽게 풀어쓴 자료구조,"천인국, 공용해, 하상호",생능출판사,"29,000",2,"C언어로 쉽게 풀어쓴 자료구조 천인국, 공용해, 하상호"
4,NaN,IT,데이터통신과 네트워킹,Forouzan Behrouz A.,McGraw-Hill,"39,000",2,데이터통신과 네트워킹 Forouzan Behrouz A.
...,...,...,...,...,...,...,...,...
118,NaN,IT,Do it! 점프 투 파이썬,박응용,이지스퍼블리싱,"18,800",1,Do it! 점프 투 파이썬 박응용
119,NaN,경제경영,인공지능 비즈니스 트렌드,네크니들,와이즈랩,"18,000",1,인공지능 비즈니스 트렌드 네크니들
120,NaN,철학,인공지능의 윤리학,"이중원,고인석,이영의 외",한울아카데미,"24,000",1,"인공지능의 윤리학 이중원,고인석,이영의 외"
121,NaN,경제경영,예측기계 (인공지능의 간단한 경제경영학),"어제이 애그러월, 조슈아 갠스, 아비 골드파브",생각의힘,"18,000",1,"예측기계 (인공지능의 간단한 경제경영학) 어제이 애그러월, 조슈아 갠스, 아비 골드파브"


link 칼럼을 붙여줍니다.

In [64]:
book_link_lst = []
for book_name in booklist['key_value']:
    req = requests.get('https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=All&SearchWord='+ book_name +'&x=0&y=0')
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    book_link = [link.get('href') for link in soup.select('div > ul > li:nth-child(2) > a')][-2]
    book_link_lst.append(book_link)
book_link_lst

['https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=101470689',
 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=364662',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 '/search/wsearchresult.aspx?PublisherSearch=Mcgraw-Hill+College@22543&BranchType=7',
 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=105851102',
 'http://www.aladin.co.kr/shop/common/wseriesitem.aspx?SRID=13045',
 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=182410060',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=6960708',
 '/shop/book/Recommend2010/SpecialEditor.aspx',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17280',
 'https://www.aladin.c

In [66]:
len(book_link_lst)

123